# ToDo :

* ~~consolidate code from notebook into WISDM helper methods file~~
* rewrite kfolds process so that we have more precise control over the size of a fold (as oppose to just the number of folds)
* run experiment that directly compares model using ALL general data + active data to model using ONLY general data from nearest cluster + active data
* analyze/visualize clusters (is there a better algorithm? is there a better k for the k-means?) 
* perhaps compare with using the WORST cluster, or using ONLY the personal data
    * for each size of active data
        * for each algorithm (also ensemble algorithm?)
            * personal only
            * universal only
            * personal + ALL universal
            * personal + best cluster universal
            * personal + worst cluster universal


In [1]:
from wisdm import wisdm
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.cluster import KMeans
from scipy.stats import mode

### Personal Note : Consider adding wget commands here to download the WISDM data

In [2]:
wisdm_1_dir = "../../WISDM_ar_v1.1/"
wisdm_1_transformed_data  = "WISDM_ar_v1.1_transformed_FIXED.arff"
wisdm_2_dir = "../../WISDM_at_v2.0/"
wisdm_2_transformed_data = "WISDM_at_v2.0_transformed_FIXED.arff"

In [3]:
wisdm.WISDM_DIR = wisdm_1_dir
wisdm.WISDM_TRANSFORMED = wisdm_1_transformed_data

In [4]:
with open("../../WISDM_ar_v1.1/WISDM_ar_v1.1_transformed.arff", "r") as fIn:
    old_lines = fIn.readlines()
    new_lines = []
    for ind, line in enumerate(old_lines):
        if ind is 3:
            bracket_index = line.find("{")
            new_line = line[:bracket_index]
            new_line += line[bracket_index:].replace('"', '')
        elif ind is 47:
            bracket_index = line.find("{")
            new_line = line[:bracket_index]
            new_line += " "+line[bracket_index:].replace('"', '')
        else:
            new_line = line
        new_lines.append(new_line)

with open("../../WISDM_ar_v1.1/WISDM_ar_v1.1_transformed_FIXED.arff", "w") as fOut:
    for line in new_lines:
        fOut.write(line)     

In [5]:
# FIXING WISDM DATA SO IT CAN LOAD with loadarff()
with open("../../WISDM_at_v2.0/WISDM_at_v2.0_transformed.arff", "r") as fIn:
    old_lines = fIn.readlines()
    new_lines = []
    for ind, line in enumerate(old_lines):
        if ind is 2:
            brack_index = line.find("{")
            new_line = line[:bracket_index]
            new_line += line[bracket_index:].replace('"', '')
        elif ind is 46:
            bracket_index = line.find("{")
            new_line = line[:bracket_index]
            new_line += " "+line[bracket_index:].replace('"', '')
        else:
            new_line = line
        new_lines.append(new_line)

with open("../../WISDM_at_v2.0/WISDM_at_v2.0_transformed_FIXED.arff", "w") as fOut:
    for line in new_lines:
        fOut.write(line)     

In [6]:
wisdm.set_data()

In [ ]:
import importlib
importlib.reload(wisdm)

# Test Universal Impersonal Model Pipeline

In [7]:
import warnings

In [8]:
import time
start = time.time()

In [9]:
rf_results = []

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for user_id in wisdm.user_ids:
        test_set = wisdm.get_user_set(user_id)
        test_set = wisdm.remove_all_nan(test_set)

        test_labels = np.array([t.decode("utf-8") for t in test_set['class'].as_matrix()])
        test_features = test_set.as_matrix(columns=[test_set.columns[1:-1]])

        #print("%s labels, %s features" % (len(test_labels), len(test_features)))
        # training features & labels
        training_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
        training_set = wisdm.remove_all_nan(training_set)
        training_labels = np.array([t.decode("utf-8") for t in training_set['class'].as_matrix()])
        training_features = training_set.as_matrix(columns=[test_set.columns[1:-1]])

        # normalize features
        scaler = StandardScaler().fit(training_features)
        scaled_train_x = scaler.transform(training_features)
        scaled_test_x = scaler.transform(test_features)

        clf = wisdm.weka_RF()
        clf.fit(scaled_train_x, training_labels)
        predictions = clf.predict(scaled_test_x)

        score = accuracy_score(test_labels, predictions)
        print("Score for %s: %.3f" % (user_id, score))
        rf_results.append(score)

print("RF results : M=%.5f, SD=%.5f" % (np.mean(rf_results), np.std(rf_results)))

Score for 33: 0.779
Score for 17: 0.337
Score for 20: 0.810
Score for 29: 0.792
Score for 13: 0.840
Score for 15: 0.917
Score for 6: 0.905
Score for 27: 0.695
Score for 36: 0.719
Score for 18: 0.845
Score for 32: 0.793
Score for 35: 0.504
Score for 11: 0.778
Score for 16: 0.755
Score for 5: 0.871
Score for 10: 0.765
Score for 28: 0.860
Score for 26: 0.794
Score for 14: 0.759
Score for 24: 0.798
Score for 12: 0.854
Score for 23: 0.744
Score for 4: 0.673
Score for 30: 0.152
Score for 34: 0.845
Score for 8: 0.978
Score for 31: 0.823
Score for 21: 0.710
Score for 3: 0.813
Score for 22: 0.832
Score for 1: 0.779
Score for 9: 0.250
Score for 25: 0.924
Score for 2: 0.523
Score for 7: 0.850
Score for 19: 0.861
RF results : M=0.74800, SD=0.17894


In [10]:
finish = time.time()
print("Took about %s seconds" % (finish - start))

Took about 19.46033477783203 seconds


# Parallelized Universal Model

In [11]:
import ipyparallel as ipp

In [12]:
c = ipp.Client()

In [13]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [14]:
dview = c[:]

In [15]:
def universal_model():
    rf_results = []

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for user_id in user_ids:
            test_set = wisdm.get_user_set(user_id)
            test_set = wisdm.remove_all_nan(test_set)

            test_labels = np.array([t.decode("utf-8") for t in test_set['class'].as_matrix()])
            test_features = test_set.as_matrix(columns=[test_set.columns[1:-1]])

            #print("%s labels, %s features" % (len(test_labels), len(test_features)))
            # training features & labels
            training_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
            training_set = wisdm.remove_all_nan(training_set)
            training_labels = np.array([t.decode("utf-8") for t in training_set['class'].as_matrix()])
            training_features = training_set.as_matrix(columns=[test_set.columns[1:-1]])

            # normalize features
            scaler = StandardScaler().fit(training_features)
            scaled_train_x = scaler.transform(training_features)
            scaled_test_x = scaler.transform(test_features)

            clf = wisdm.weka_RF()
            clf.fit(scaled_train_x, training_labels)
            predictions = clf.predict(scaled_test_x)

            score = accuracy_score(test_labels, predictions)
            print("Score for %s: %.3f" % (user_id, score))
            rf_results.append(score)

    print("RF results : M=%.5f, SD=%.5f" % (np.mean(rf_results), np.std(rf_results)))
    return rf_results

In [16]:
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

In [17]:
dview['user_ids']

[array(['33', '17', '20'], dtype=object),
 array(['29', '13', '15'], dtype=object),
 array(['6', '27', '36'], dtype=object),
 array(['18', '32', '35'], dtype=object),
 array(['11', '16', '5'], dtype=object),
 array(['10', '28', '26'], dtype=object),
 array(['14', '24'], dtype=object),
 array(['12', '23'], dtype=object),
 array(['4', '30'], dtype=object),
 array(['34', '8'], dtype=object),
 array(['31', '21'], dtype=object),
 array(['3', '22'], dtype=object),
 array(['1', '9'], dtype=object),
 array(['25', '2'], dtype=object),
 array(['7', '19'], dtype=object)]

In [18]:
def add_wisdm_helpers_path():
    import sys
    sys.path.append("/home/sac086/hcml_chapter/wisdm_model_personalization/notebooks/")

In [19]:
dview.apply_async(add_wisdm_helpers_path)

<AsyncResult: add_wisdm_helpers_path>

In [20]:
%%px --local
import warnings
import os
from wisdm import wisdm
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.cluster import KMeans
from scipy.stats import mode

In [21]:
with dview.sync_imports():
    import warnings
    import os
    from wisdm import wisdm
    import numpy as np
    import pandas as pd

    from sklearn.metrics import accuracy_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
    from sklearn.cluster import KMeans
    from scipy.stats import mode

importing warnings on engine(s)
importing os on engine(s)
importing wisdm from wisdm on engine(s)
importing numpy on engine(s)
importing pandas on engine(s)
importing accuracy_score from sklearn.metrics on engine(s)
importing StandardScaler from sklearn.preprocessing on engine(s)
importing StratifiedKFold,StratifiedShuffleSplit from sklearn.model_selection on engine(s)
importing KMeans from sklearn.cluster on engine(s)
importing mode from scipy.stats on engine(s)


In [22]:
dview.block=True

In [23]:
def init():
    wisdm_1_dir = "/home/sac086/hcml_chapter/WISDM_ar_v1.1/"
    wisdm_1_transformed_data  = "WISDM_ar_v1.1_transformed_FIXED.arff"
    wisdm_2_dir = "/home/sac086/hcml_chapter/WISDM_at_v2.0/"
    wisdm_2_transformed_data = "WISDM_at_v2.0_transformed_FIXED.arff"
    
    wisdm.WISDM_DIR = wisdm_1_dir
    wisdm.WISDM_TRANSFORMED = wisdm_1_transformed_data
    
    wisdm.set_data()
    print("Done")

In [24]:
dview.apply(init)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [25]:
start = time.time()

In [26]:
dview.apply(universal_model)

[[0.74496644295302017, 0.40384615384615385, 0.78306878306878303],
 [0.7865168539325843, 0.84571428571428575, 0.91666666666666663],
 [0.91216216216216217, 0.67682926829268297, 0.69784172661870503],
 [0.85161290322580641, 0.7931034482758621, 0.5663716814159292],
 [0.76388888888888884, 0.80392156862745101, 0.86428571428571432],
 [0.76470588235294112, 0.90654205607476634, 0.79393939393939394],
 [0.69109947643979053, 0.79838709677419351],
 [0.84552845528455289, 0.73643410852713176],
 [0.71153846153846156, 0.15178571428571427],
 [0.8214285714285714, 0.97810218978102192],
 [0.83732057416267947, 0.62616822429906538],
 [0.8193548387096774, 0.84955752212389379],
 [0.76551724137931032, 0.203125],
 [0.9242424242424242, 0.54128440366972475],
 [0.82677165354330706, 0.86082474226804129]]

In [27]:
finish = time.time()
print("Finished in %s seconds" % (finish-start))

Finished in 5.647056818008423 seconds


# Cross-Validation with exact number of data points

### Experiment Attributes : 
* amount of training set from individual end user
* amount of training set from impersonal data (other users)
    * "ALL" all other data
    * "closest cluster" only data from the closest cluster
    * "furthest cluster" only data from the furthest cluster
    * "All - furthest cluster" all other data EXCEPT data from furthest cluster
* test user id
* algorithm
* algorithm parameters



In [28]:
%%px --local
def personal_model(active_features, active_labels, test_features, test_labels):
    scaler = StandardScaler().fit(active_features)
    scaled_train_x = scaler.transform(active_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()
    rfc_clf.fit(scaled_train_x, active_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)
    return score

In [29]:
%%px --local

def universal_model(universal_features, universal_labels, test_features, test_labels):
    scaler = StandardScaler().fit(universal_features)
    scaled_train_x = scaler.transform(universal_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()
    rfc_clf.fit(scaled_train_x, universal_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)

    return score

In [30]:
%%px --local

def universal_plus_personal_model(personal_features, personal_labels,
                                  universal_features, universal_labels,
                                  test_features, test_labels):
    personal_plus_universal_features = np.vstack((personal_features, universal_features))
    personal_plus_universal_labels = np.hstack((personal_labels, universal_labels))

    scaler = StandardScaler().fit(personal_plus_universal_features)
    scaled_train_x = scaler.transform(personal_plus_universal_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()

    rfc_clf.fit(scaled_train_x, personal_plus_universal_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)
    return score

In [31]:
%%px --local

def cluster_plus_personal_model(personal_features, personal_labels,
                                  universal_features, universal_labels,
                                  test_features, test_labels, KM):
    cluster_predictions = KM.predict(personal_features)
    closest_cluster = mode(cluster_predictions).mode[0]

    cluster_data_indeces = [i for i in range(len(clusters)) if clusters[i] == closest_cluster]
    cluster_features = universal_features[cluster_data_indeces]
    cluster_labels = universal_labels[cluster_data_indeces]

    training_features = np.vstack((personal_features, cluster_features))
    training_labels = np.hstack((personal_labels, cluster_labels))

    scaler = StandardScaler().fit(training_features)
    scaled_train_x = scaler.transform(training_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()

    rfc_clf.fit(scaled_train_x, training_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)
    return score

In [32]:
%%px --local

from collections import Counter

In [33]:
training_sizes = [10,20,30,40,50,60,70,80,90,100]

def all_models():
    scores = []
    err = None
    training_sizes = [10,20,30,40,50,60,70,80,90,100]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for ind, user_id in enumerate(user_ids):
            user_scores_df = []
            print("Running user #%s: %s" % (ind, user_id))
            personal_set = wisdm.get_user_set(user_id)
            personal_set = wisdm.remove_all_nan(personal_set)

            personal_labels = np.array([t.decode("utf-8") for t in personal_set['class'].as_matrix()])
            personal_features = personal_set.as_matrix(columns=[personal_set.columns[1:-1]])

            # What is the distribution of labels for this participant?
            personal_labels_distribution = Counter(personal_labels)
            print("\tHas %s labels : " % len(personal_labels))
            for label_key, number in personal_labels_distribution.items():
                print("\t\t%s:%s" % (label_key, number))
            print("\n")
            # training features & labels
            universal_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
            universal_set = wisdm.remove_all_nan(universal_set)
            universal_labels = np.array([t.decode("utf-8") for t in universal_set['class'].as_matrix()])
            universal_features = universal_set.as_matrix(columns=[universal_set.columns[1:-1]])

            # get k-means clusters
            number_of_clusters = 4 # the higher this number is, the smaller we should expect each cluster to be

            KM = KMeans(n_clusters=number_of_clusters)
            clusters = KM.fit_predict(universal_features)

            if clusters:
                return clusters
            k = 10

            skf = StratifiedKFold(n_splits=k)

            k_run = 0
            for active_index, test_index in skf.split(personal_features, personal_labels):
                print("\tRunning Fold #%s\n" % k_run)
                # data set available for active labeling from the individual
                all_active_features = personal_features[active_index]
                all_active_labels = personal_labels[active_index]


                # held out test set from individual
                test_features = personal_features[test_index]
                test_labels = personal_labels[test_index]

                # iterate through size of training data
                for training_size in training_sizes:
                    # initialize score holders
                    personal_model_scores = []
                    universal_model_scores = []
                    personal_plus_all_scores = []
                    personal_plus_cluster_scores = []

                    # run universal model
                    universal_model_score = universal_model(universal_features, universal_labels,
                                                            test_features, test_labels)
                    universal_model_scores.append(universal_model_score)

                    sss = StratifiedShuffleSplit(n_splits=5, train_size=training_size, test_size=6)

                    splits = sss.split(all_active_features, all_active_labels)

                    try:
                        for split_num, split_tup in enumerate(splits):
                            sampled_active_index, __ = split_tup
                            sampled_active_features = personal_features[sampled_active_index]
                            sampled_active_labels = personal_labels[sampled_active_index]

                            # run personal model
                            personal_score = personal_model(sampled_active_features, sampled_active_labels, test_features, test_labels)
                            personal_model_scores.append(personal_score)

                            # run personal + universal
                            personal_plus_all_score = universal_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                    universal_features, universal_labels,
                                                                                    test_features, test_labels)
                            personal_plus_all_scores.append(personal_plus_all_score)

                            # run personal + cluster
                            personal_plus_cluster_score = cluster_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                    universal_features, universal_labels,
                                                                                    test_features, test_labels, KM)
                            personal_plus_cluster_scores.append(personal_plus_cluster_score)
                    except ValueError as ve:
                        print("Error with training size during split #%s" % split_num)
                        print("Message : %s" % ve.args[0])
                        err = ve
                        if "Reduce test_size and/or train_size" in ve.args[0]:
                            print("continuing...")
                            continue
                        elif "should be smaller than the number of samples" in ve.args[0]:
                            print("continuing...")
                            continue
                        elif "The least populated class in y has only 1 member" in ve.args[0]:
                            print("continuing...")
                            continue
                        else:
                            raise(ve)

                    row = {"test user" : user_id,
                           "k-run" : k_run,
                       "classifier" : "RF with Wiki Parameters",
                       "personal training data" : training_size,
                       "personal score Mean" : np.mean(personal_model_scores),
                       "personal score STD" : np.std(personal_model_scores),
                       "impersonal score Mean" : np.mean(universal_model_scores),
                       "impersonal score STD" : np.std(universal_model_scores),
                       "personal + impersonal score Mean" : np.mean(personal_plus_all_scores),
                       "personal + impersonal score STD" : np.std(personal_plus_all_scores),
                       "personal + cluster score Mean" : np.mean(personal_plus_cluster_scores),
                       "personal + cluster score STD" : np.std(personal_plus_cluster_scores)
                       }
                    print("\tamount of personal data : %s row" % training_size)
                    print("\tpersonal model score : M=%.3f, SD=%.3f" % (row["personal score Mean"], row["personal score STD"]))
                    print("\tuniversal model score : M=%.3f, SD=%.3f" % (row["impersonal score Mean"], row["impersonal score STD"]))
                    print("\tpersonal + ALL universal : M=%.3f, SD=%.3f" % (row["personal + impersonal score Mean"], row["personal + impersonal score STD"]))
                    print("\tpersonal + CLUSTER universal : M=%.3f, SD=%.3f" % (row["personal + cluster score Mean"], row["personal + cluster score STD"]))
                    print("\n")
                    scores.append(row)
                    user_scores_df.append(row)
                k_run += 1
            user_scores_df = pd.DataFrame(user_scores_df)
            user_scores_df.to_pickle("/home/sac086/hcml_chapter/wisdm_model_personalization/notebooks/results/experiment_08-17/"+user_id+".pickle")

    scores_df = pd.DataFrame(scores)

In [34]:
start = time.time()

In [35]:
results = dview.apply(all_models)

CompositeError: one or more exceptions from call to method: all_models
[0:apply]: ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[1:apply]: ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[2:apply]: ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
[3:apply]: ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
.... 11 more exceptions ...

In [ ]:
finish = time.time()
print("Finished all models in %s seconds" % (finish - start))

# Paralellizing to run on WISDM_2.0

In [ ]:
training_sizes = [10,20,30,40,50,60,70,80,90,100]

scores = []
err = None
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for ind, user_id in enumerate(wisdm.user_ids):
        user_scores_df = []
        print("Running user #%s: %s" % (ind, user_id))
        personal_set = wisdm.get_user_set(user_id)
        personal_set = wisdm.remove_all_nan(personal_set)

        personal_labels = np.array([t.decode("utf-8") for t in personal_set['class'].as_matrix()])
        personal_features = personal_set.as_matrix(columns=[personal_set.columns[1:-1]])
        
        # What is the distribution of labels for this participant?
        personal_labels_distribution = Counter(personal_labels)
        print("\tHas %s labels : " % len(personal_labels))
        for label_key, number in personal_labels_distribution.items():
            print("\t\t%s:%s" % (label_key, number))
        print("\n")
        # training features & labels
        universal_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
        universal_set = wisdm.remove_all_nan(universal_set)
        universal_labels = np.array([t.decode("utf-8") for t in universal_set['class'].as_matrix()])
        universal_features = universal_set.as_matrix(columns=[universal_set.columns[1:-1]])
        
        # get k-means clusters
        number_of_clusters = 4 # the higher this number is, the smaller we should expect each cluster to be

        KM = KMeans(n_clusters=number_of_clusters)
        clusters = KM.fit_predict(universal_features)
        
        k = 10
        
        skf = StratifiedKFold(n_splits=k)
        
        k_run = 0
        for active_index, test_index in skf.split(personal_features, personal_labels):
            print("\tRunning Fold #%s\n" % k_run)
            # data set available for active labeling from the individual
            all_active_features = personal_features[active_index]
            all_active_labels = personal_labels[active_index]


            # held out test set from individual
            test_features = personal_features[test_index]
            test_labels = personal_labels[test_index]

            # iterate through size of training data
            for training_size in training_sizes:
                # initialize score holders
                personal_model_scores = []
                universal_model_scores = []
                personal_plus_all_scores = []
                personal_plus_cluster_scores = []

                # run universal model
                universal_model_score = universal_model(universal_features, universal_labels,
                                                        test_features, test_labels)
                universal_model_scores.append(universal_model_score)

                sss = StratifiedShuffleSplit(n_splits=5, train_size=training_size, test_size=6)

                splits = sss.split(all_active_features, all_active_labels)

                try:
                    for split_num, split_tup in enumerate(splits):
                        sampled_active_index, __ = split_tup
                        sampled_active_features = personal_features[sampled_active_index]
                        sampled_active_labels = personal_labels[sampled_active_index]

                        # run personal model
                        personal_score = personal_model(sampled_active_features, sampled_active_labels, test_features, test_labels)
                        personal_model_scores.append(personal_score)

                        # run personal + universal
                        personal_plus_all_score = universal_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                universal_features, universal_labels,
                                                                                test_features, test_labels)
                        personal_plus_all_scores.append(personal_plus_all_score)

                        # run personal + cluster
                        personal_plus_cluster_score = cluster_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                universal_features, universal_labels,
                                                                                test_features, test_labels, KM)
                        personal_plus_cluster_scores.append(personal_plus_cluster_score)
                except ValueError as ve:
                    print("Error with training size during split #%s" % split_num)
                    print("Message : %s" % ve.args[0])
                    err = ve
                    if "Reduce test_size and/or train_size" in ve.args[0]:
                        print("continuing...")
                        continue
                    elif "should be smaller than the number of samples" in ve.args[0]:
                        print("continuing...")
                        continue
                    elif "The least populated class in y has only 1 member" in ve.args[0]:
                        print("continuing...")
                        continue
                    else:
                        raise(ve)

                row = {"test user" : user_id,
                       "k-run" : k_run,
                   "classifier" : "RF with Wiki Parameters",
                   "personal training data" : training_size,
                   "personal score Mean" : np.mean(personal_model_scores),
                   "personal score STD" : np.std(personal_model_scores),
                   "impersonal score Mean" : np.mean(universal_model_scores),
                   "impersonal score STD" : np.std(universal_model_scores),
                   "personal + impersonal score Mean" : np.mean(personal_plus_all_scores),
                   "personal + impersonal score STD" : np.std(personal_plus_all_scores),
                   "personal + cluster score Mean" : np.mean(personal_plus_cluster_scores),
                   "personal + cluster score STD" : np.std(personal_plus_cluster_scores)
                   }
                print("\tamount of personal data : %s row" % training_size)
                print("\tpersonal model score : M=%.3f, SD=%.3f" % (row["personal score Mean"], row["personal score STD"]))
                print("\tuniversal model score : M=%.3f, SD=%.3f" % (row["impersonal score Mean"], row["impersonal score STD"]))
                print("\tpersonal + ALL universal : M=%.3f, SD=%.3f" % (row["personal + impersonal score Mean"], row["personal + impersonal score STD"]))
                print("\tpersonal + CLUSTER universal : M=%.3f, SD=%.3f" % (row["personal + cluster score Mean"], row["personal + cluster score STD"]))
                print("\n")
                scores.append(row)
                user_scores_df.append(row)
            k_run += 1
        user_scores_df = pd.DataFrame(user_scores_df)
        user_scores_df.to_pickle("./results/experiment_08-17/"+user_id+".pickle")

scores_df = pd.DataFrame(scores)